In [1]:
import pandas as pd
from itertools import combinations
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
co_occurrence = defaultdict(int)
character_counts = defaultdict(int)

In [3]:
df = pd.read_csv('Friends.csv')

In [4]:
df = df.dropna(subset=['Speaker'])

In [5]:
df['Speaker'] = df['Speaker'].str.title()

In [6]:
unique_speakers = df['Speaker'].unique()

In [7]:
top_30_characters = df['Speaker'].value_counts().head(30)

print(top_30_characters)

Ross              8869
Rachel            8774
Chandler          8058
Joey              8047
Monica            7965
Phoebe            7152
[Scene            2833
Mike               355
All                332
Rach               325
Mnca               259
Richard            254
Chan               233
Janice             208
Mr. Geller         204
Phoe               195
Carol              192
Charlie            190
Transcribed By     171
Mrs. Geller        167
Emily              167
Tag                146
Frank              133
Paul               130
Gunther            127
Written By         126
David              120
Amy                116
Mona               111
Woman              105
Name: Speaker, dtype: int64


In [8]:
entries_to_remove = ["[Scene", "All", "Transcribed By", "Written By", "Woman"]

In [9]:
df = df[~df['Speaker'].isin(entries_to_remove)]

In [10]:
replacements = {
    r'\bMnca\b': 'Monica',
    r'\bRach\b': 'Rachel',
    r'\bPhoe\b': 'Phoebe',
    r'\bChan\b': 'Chandler'
}

df['Speaker'] = df['Speaker'].replace(replacements, regex=True)

In [11]:
top_30_characters = df['Speaker'].value_counts().head(30)

print(top_30_characters)

Rachel         9099
Ross           8869
Chandler       8291
Monica         8224
Joey           8047
Phoebe         7347
Mike            355
Richard         254
Janice          208
Mr. Geller      204
Carol           192
Charlie         190
Emily           167
Mrs. Geller     167
Tag             146
Frank           133
Paul            130
Gunther         127
David           120
Amy             116
Mona            111
Pete            103
Susan           102
Joshua           98
Gary             96
Elizabeth        94
Janine           92
Kathy            91
Jill             83
Ben              73
Name: Speaker, dtype: int64


In [12]:
top_30_characters_list = top_30_characters.index.tolist()

In [13]:
df = df[df['Speaker'].isin(top_30_characters_list)]

In [14]:
print(df['Speaker'].unique())

['Monica' 'Joey' 'Chandler' 'Phoebe' 'Ross' 'Rachel' 'Paul' 'Carol'
 'Mrs. Geller' 'Mr. Geller' 'Susan' 'Jill' 'David' 'Janice' 'Gunther'
 'Richard' 'Ben' 'Frank' 'Mike' 'Pete' 'Kathy' 'Joshua' 'Emily'
 'Elizabeth' 'Gary' 'Janine' 'Tag' 'Mona' 'Amy' 'Charlie']


In [15]:
co_occurrence = defaultdict(int)
character_counts = defaultdict(int)

In [16]:
for _, episode_group in df.groupby(['Season', 'Episode']):
    characters = episode_group['Speaker'].unique()
    for character in characters:
        character_counts[character] += 1
    for char_a, char_b in combinations(characters, 2):
        co_occurrence[frozenset([char_a, char_b])] += 1

In [17]:
nodes = [{"Name": character, "Weight": count} for character, count in character_counts.items()]
edges = []

for characters, count in co_occurrence.items():
    char_a, char_b = characters
    edges.append({
        "source": char_a,
        "target": char_b,
        "weight": count,
        "sourceIndex": list(character_counts.keys()).index(char_a),
        "targetIndex": list(character_counts.keys()).index(char_b)
    })

In [18]:
print(nodes)

[{'Name': 'Monica', 'Weight': 225}, {'Name': 'Joey', 'Weight': 225}, {'Name': 'Chandler', 'Weight': 225}, {'Name': 'Phoebe', 'Weight': 225}, {'Name': 'Ross', 'Weight': 225}, {'Name': 'Rachel', 'Weight': 225}, {'Name': 'Paul', 'Weight': 4}, {'Name': 'Carol', 'Weight': 15}, {'Name': 'Mrs. Geller', 'Weight': 17}, {'Name': 'Mr. Geller', 'Weight': 19}, {'Name': 'Susan', 'Weight': 11}, {'Name': 'Jill', 'Weight': 4}, {'Name': 'David', 'Weight': 5}, {'Name': 'Janice', 'Weight': 18}, {'Name': 'Gunther', 'Weight': 51}, {'Name': 'Richard', 'Weight': 9}, {'Name': 'Ben', 'Weight': 10}, {'Name': 'Frank', 'Weight': 6}, {'Name': 'Mike', 'Weight': 18}, {'Name': 'Pete', 'Weight': 6}, {'Name': 'Kathy', 'Weight': 7}, {'Name': 'Joshua', 'Weight': 5}, {'Name': 'Emily', 'Weight': 12}, {'Name': 'Elizabeth', 'Weight': 6}, {'Name': 'Gary', 'Weight': 5}, {'Name': 'Janine', 'Weight': 6}, {'Name': 'Tag', 'Weight': 7}, {'Name': 'Mona', 'Weight': 7}, {'Name': 'Amy', 'Weight': 2}, {'Name': 'Charlie', 'Weight': 8}]


In [19]:
print(edges)

[{'source': 'Monica', 'target': 'Joey', 'weight': 225, 'sourceIndex': 0, 'targetIndex': 1}, {'source': 'Monica', 'target': 'Chandler', 'weight': 225, 'sourceIndex': 0, 'targetIndex': 2}, {'source': 'Monica', 'target': 'Phoebe', 'weight': 225, 'sourceIndex': 0, 'targetIndex': 3}, {'source': 'Monica', 'target': 'Ross', 'weight': 225, 'sourceIndex': 0, 'targetIndex': 4}, {'source': 'Monica', 'target': 'Rachel', 'weight': 225, 'sourceIndex': 0, 'targetIndex': 5}, {'source': 'Monica', 'target': 'Paul', 'weight': 4, 'sourceIndex': 0, 'targetIndex': 6}, {'source': 'Joey', 'target': 'Chandler', 'weight': 225, 'sourceIndex': 1, 'targetIndex': 2}, {'source': 'Phoebe', 'target': 'Joey', 'weight': 225, 'sourceIndex': 3, 'targetIndex': 1}, {'source': 'Joey', 'target': 'Ross', 'weight': 225, 'sourceIndex': 1, 'targetIndex': 4}, {'source': 'Joey', 'target': 'Rachel', 'weight': 225, 'sourceIndex': 1, 'targetIndex': 5}, {'source': 'Joey', 'target': 'Paul', 'weight': 4, 'sourceIndex': 1, 'targetIndex': 

In [20]:
final_dataset = {
    "nodes": nodes,
    "edges": edges
}

In [21]:
print(final_dataset)

{'nodes': [{'Name': 'Monica', 'Weight': 225}, {'Name': 'Joey', 'Weight': 225}, {'Name': 'Chandler', 'Weight': 225}, {'Name': 'Phoebe', 'Weight': 225}, {'Name': 'Ross', 'Weight': 225}, {'Name': 'Rachel', 'Weight': 225}, {'Name': 'Paul', 'Weight': 4}, {'Name': 'Carol', 'Weight': 15}, {'Name': 'Mrs. Geller', 'Weight': 17}, {'Name': 'Mr. Geller', 'Weight': 19}, {'Name': 'Susan', 'Weight': 11}, {'Name': 'Jill', 'Weight': 4}, {'Name': 'David', 'Weight': 5}, {'Name': 'Janice', 'Weight': 18}, {'Name': 'Gunther', 'Weight': 51}, {'Name': 'Richard', 'Weight': 9}, {'Name': 'Ben', 'Weight': 10}, {'Name': 'Frank', 'Weight': 6}, {'Name': 'Mike', 'Weight': 18}, {'Name': 'Pete', 'Weight': 6}, {'Name': 'Kathy', 'Weight': 7}, {'Name': 'Joshua', 'Weight': 5}, {'Name': 'Emily', 'Weight': 12}, {'Name': 'Elizabeth', 'Weight': 6}, {'Name': 'Gary', 'Weight': 5}, {'Name': 'Janine', 'Weight': 6}, {'Name': 'Tag', 'Weight': 7}, {'Name': 'Mona', 'Weight': 7}, {'Name': 'Amy', 'Weight': 2}, {'Name': 'Charlie', 'Weigh

In [22]:
import json
with open('friends-cooccur.json', 'w') as f:
    json.dump(final_dataset, f, indent=4)

In [23]:
df['Season'] = df['Season'].str.extract(r'(\d+)').astype(int)
df['Episode'] = df['Episode'].str.extract(r'(\d+)').astype(int)

In [24]:
df["Text"]

3         There's nothing to tell! He's just some guy I...
4         C'mon, you're going out with the guy! There's...
5          So does he have a hump? A hump and a hairpiece?
6                                 Wait, does he eat chalk?
8         Just, 'cause, I don't want her to go through ...
                               ...                        
69966           Yeah! I'm going to Paris. Thank you, Ross!
69967                           Yeah, yeah, oh! (They hug)
69968                                Oh! Oh, I'm so happy.
69969     Then I'm happy too. (They're still hugging - ...
69972     Thank you all for coming. We're here today to...
Name: Text, Length: 53329, dtype: object

In [25]:
df['WordCount'] = df['Text'].apply(lambda text: len(str(text).split()))

In [26]:
word_count_by_season = df.groupby(['Speaker', 'Season']).WordCount.sum().reset_index()

In [27]:
print(word_count_by_season)

    Speaker  Season  WordCount
0       Amy       9        720
1       Amy      10        721
2       Ben       2          5
3       Ben       3          8
4       Ben       5          9
..      ...     ...        ...
150   Susan       3         35
151   Susan       4         70
152   Susan       6         36
153     Tag       7       1157
154     Tag       8        110

[155 rows x 3 columns]


In [28]:
unique_characters = df['Speaker'].unique()
unique_seasons = df['Season'].unique()

In [29]:
print(unique_characters)

['Monica' 'Joey' 'Chandler' 'Phoebe' 'Ross' 'Rachel' 'Paul' 'Carol'
 'Mrs. Geller' 'Mr. Geller' 'Susan' 'Jill' 'David' 'Janice' 'Gunther'
 'Richard' 'Ben' 'Frank' 'Mike' 'Pete' 'Kathy' 'Joshua' 'Emily'
 'Elizabeth' 'Gary' 'Janine' 'Tag' 'Mona' 'Amy' 'Charlie']


In [30]:
import itertools
all_combinations = pd.DataFrame(list(itertools.product(unique_characters, unique_seasons)), columns=['Speaker', 'Season'])

In [31]:
word_count_by_season = df.groupby(['Speaker', 'Season']).WordCount.sum().reset_index()

complete_data = all_combinations.merge(word_count_by_season, on=['Speaker', 'Season'], how='left').fillna(0)

complete_data['WordCount'] = complete_data['WordCount'].astype(int)

In [32]:
complete_data

,Speaker,Season,WordCount
0,Monica,1,8004
1,Monica,2,8207
2,Monica,3,8669
3,Monica,4,8217
4,Monica,5,9806
...,...,...,...
295,Charlie,6,0
296,Charlie,7,0
297,Charlie,8,0
298,Charlie,9,1428


In [33]:
complete_data.to_csv('word-count-per-season.csv', index=False)

In [34]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /Users/emir/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /Users/emir/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/emir/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /Users/emir/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Most Common Words

In [36]:
from nltk.tokenize import word_tokenize

In [37]:
lemmatizer = WordNetLemmatizer()

In [38]:
def preprocess_text(text):
    words = word_tokenize(text.lower())
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word.isalpha() and word not in stop_words]
    return filtered_words

In [39]:
df['ProcessedWords'] = df['Text'].apply(preprocess_text)

In [40]:
from collections import Counter

word_counts_by_character = df.groupby('Speaker')['ProcessedWords'].sum().apply(Counter)

In [41]:
top_n = 10
data = []

for speaker, counter in word_counts_by_character.items():
    top_words_counts = counter.most_common(top_n)
    speaker_data = {"Speaker": speaker}
    speaker_data.update({f"Word_{i+1}": word for i, (word, _) in enumerate(top_words_counts)})
    speaker_data.update({f"Count_{i+1}": count for i, (_, count) in enumerate(top_words_counts)})
    data.append(speaker_data)

In [42]:
words_df = pd.DataFrame(data)

In [43]:
most_common_words_data = []

for _, row in words_df.iterrows():
    speaker = row['Speaker']
    words_list = [{'word': row[f'Word_{i+1}'], 'count': row[f'Count_{i+1}']} for i in range(top_n)]
    most_common_words_data.append({'character': speaker, 'words': words_list})

### Most Distinguishing Words

In [44]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [45]:
df['Words'] = df['Text'].apply(lambda x: nltk.word_tokenize(x.lower()))
df['Words'] = df['Words'].apply(lambda x: [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in x if word.isalpha() and word not in stop_words])

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [47]:
df['Words'] = df['Text'].apply(lambda x: nltk.word_tokenize(x.lower()))

In [48]:
df['Words']

3        [there, 's, nothing, to, tell, !, he, 's, just...
4        [c'mon, ,, you, 're, going, out, with, the, gu...
5        [so, does, he, have, a, hump, ?, a, hump, and,...
6                       [wait, ,, does, he, eat, chalk, ?]
8        [just, ,, 'cause, ,, i, do, n't, want, her, to...
                               ...                        
69966    [yeah, !, i, 'm, going, to, paris, ., thank, y...
69967           [yeah, ,, yeah, ,, oh, !, (, they, hug, )]
69968                  [oh, !, oh, ,, i, 'm, so, happy, .]
69969    [then, i, 'm, happy, too, ., (, they, 're, sti...
69972    [thank, you, all, for, coming, ., we, 're, her...
Name: Words, Length: 53329, dtype: object

In [49]:
df['Words'] = df['Words'].apply(lambda x: [word for word in x if word.isalpha() and word not in stop_words])

In [50]:
df["Words"]

3                               [nothing, tell, guy, work]
4                  [going, guy, got, ta, something, wrong]
5                                  [hump, hump, hairpiece]
6                                       [wait, eat, chalk]
8                                     [want, go, went, oh]
                               ...                        
69966                    [yeah, going, paris, thank, ross]
69967                                [yeah, yeah, oh, hug]
69968                                      [oh, oh, happy]
69969                        [happy, still, hugging, fade]
69972    [thank, coming, today, pay, respects, wonderfu...
Name: Words, Length: 53329, dtype: object

In [51]:
df['ProcessedText'] = df['Words'].apply(lambda x: ' '.join(x))

In [52]:
corpora = df.groupby('Speaker')['ProcessedText'].apply(' '.join)

tfidf_vectorizer = TfidfVectorizer(min_df=3, max_df=0.85, ngram_range=(1, 2))
tfidf_matrix = tfidf_vectorizer.fit_transform(corpora)

feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.todense(), index=corpora.index, columns=feature_names)

In [53]:
for speaker in corpora.index:
    print(f"{speaker}:")
    speaker_words = tfidf_df.loc[speaker].sort_values(ascending=False).head(10)
    print(speaker_words)

Amy:
baby                 0.228311
ross                 0.196284
ella                 0.167527
guy                  0.142752
hair straightener    0.125645
straightener         0.125645
sister               0.123330
um                   0.122817
amy                  0.116446
guys                 0.113114
Name: Amy, dtype: float64
Ben:
santa          0.566279
monica bang    0.203795
monica         0.193791
runs           0.192497
bang           0.179518
bye            0.172953
damnit         0.169884
bye bye        0.161385
daddy          0.153784
doug           0.146597
Name: Ben, dtype: float64
Carol:
susan    0.451294
ross     0.291096
ben      0.218473
cramp    0.102485
hi hi    0.100597
door     0.096728
helen    0.094981
ow       0.091290
funny    0.089715
said     0.087329
Name: Carol, dtype: float64
Chandler:
monica    0.255965
joey      0.249683
look      0.199158
ross      0.181077
yes       0.173667
ok        0.109684
man       0.108002
let       0.099863
thing     0.098782
gr

In [54]:
distinguishing_words_data = []

for speaker in corpora.index:
    speaker_words = tfidf_df.loc[speaker].sort_values(ascending=False).head(top_n)
    words_list = [{'word': word, 'count': count} for word, count in speaker_words.items()]
    distinguishing_words_data.append({'character': speaker, 'words': words_list})

### JSON Conversion

In [55]:
most_common_words_json = json.dumps(most_common_words_data)
distinguishing_words_json = json.dumps(distinguishing_words_data)

with open('most_common_words.json', 'w') as f:
    f.write(most_common_words_json)

with open('distinguishing_words.json', 'w') as f:
    f.write(distinguishing_words_json)